In [257]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import numpy as np
from urllib.parse import urlparse, parse_qs

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [260]:
URL='https://someurl.com/with/query_string?i=main&mode=front&sid=12ab&enc=+Hello'
parsed_url = urlparse(URL)
parse_qs(parsed_url.query)

{'i': ['main'], 'mode': ['front'], 'sid': ['12ab'], 'enc': [' Hello']}

In [210]:
df_hitters = pd.read_csv("2022_batters_jan4.csv")
df_starters = pd.read_csv("2022_starters_updated_feb21.csv")
df_relievers = pd.read_csv("2022_relievers_feb4.csv")

In [211]:
## Add in positions

df_positions = pd.read_csv("player_pos_data.csv")

In [212]:
merge_positions = df_positions[['idfangraphs', 'allpos']]
df_hitters_merged = df_hitters.merge(merge_positions, how='left', on='idfangraphs')
df_hitters_merged['allpos'] = df_hitters_merged['allpos'].fillna('DH')
df_hitters_merged['Position'] = df_hitters_merged['allpos'].str.split('/')
df_hitters_merged['MI'] = df_hitters_merged.apply(lambda x: ['MI'] if any(elem in x.Position  for elem in ['2B', 'SS']) else [], axis=1)
df_hitters_merged['CI'] = df_hitters_merged.apply(lambda x: ['CI'] if any(elem in x.Position  for elem in ['1B', '3B']) else [], axis=1)
df_hitters_merged['Pos'] = df_hitters_merged.apply(lambda x: x.Position + x.MI + x.CI + ['UTIL'], axis=1)


In [213]:
# Need to add in OPS, TB, RBI+R, xBH, SB-CS
df_hitters_merged['OPS'] = df_hitters_merged.apply(lambda x: x['OBP'] + x['SLG'], axis=1)
df_hitters_merged['TB'] = df_hitters_merged.apply(lambda x: x['S'] + (2 * x['D']) + (3 * x['T']) + (4 * x['HR']), axis=1)
df_hitters_merged['RBI+R'] = df_hitters_merged.apply(lambda x: x['RBI'] + x['R'], axis=1)
df_hitters_merged['xBH'] = df_hitters_merged.apply(lambda x: x['D'] + x['T'] + x['HR'], axis=1)
df_hitters_merged['SB-CS'] = df_hitters_merged.apply(lambda x: x['SB'] - x['CS'], axis=1)
df_hitters_merged['HBP'] = 0


In [235]:
league_format_options = ['Roto', 'Points']
league_format = 'Points'

# When Roto is selected you get your choice of these categories

batting_categories = ['AVG', 'RBI', 'R', 'SB', 'HR', 'OBP', 'SLG', 'OPS', 'H', 'SO', 'S', 'D', 'T', 'TB', 'BB', 'RBI+R', 'xBH', 'SB-CS', 'wOBA']
pitching_categories = ['W', 'SV', 'ERA', 'WHIP', 'K', 'AVG', 'K/9', 'BB/9', 'K/BB', 'IP', 'QS', 'HR', 'HLD', 'SV+HLD']

# Default scoring categories

selected_batting_stats = ['AVG', 'RBI', 'R', 'SB', 'HR']
selected_pitching_stats = ['W', 'SV', 'ERA', 'WHIP', 'K']

league_host = ['Yahoo', 'ESPN', 'CBS', 'Fantrax']

points_cats_batters = ['PA', 'H', 'S', 'D', 'T', 'HR', 'SO', 'BB', 'HBP', 'SB', 'CS', 'R', 'RBI']

y_points_batters = [0, 0, 2.6, 5.2, 7.8, 10.4, 0, 2.6, 2.6, 4.2, 0, 1.9, 1.9]
e_points_batters = [0, 0, 1, 2, 3, 4, -1, 1, 0, 1, 0, 1, 1]
c_points_batters = [0, 0, 1, 2, 3, 4, -.5, 1, 1, 2, -1, 1, 1]
f_points_batters = [0, 0, 1, 2, 3, 4, 0, 1, 1, 2, 0, 1, 1]

custom_points = []

custom_points = y_points_batters





In [230]:
def calculate_hitting(pa, h, b1, b2, b3, hr, so, bb, hbp, sb, cs, r, rbi, pa_points, h_points, b1_points, b2_points, b3_points, hr_points, so_points, bb_points, hbp_points, sb_points, cs_points, r_points, rbi_points):
    return ((int(pa) * pa_points) +
            (int(h) * h_points) +
            (int(b1) * b1_points) + 
            (int(b2) * b2_points) + 
            (int(b3) * b3_points) + 
            (int(hr) * hr_points) +
            (int(hbp) * hbp_points) +
            (int(sb) * sb_points) +
            (int(cs) * cs_points) +
            (int(r) * r_points) +
            (int(rbi) * rbi_points) +
            (int(bb) * bb_points) +
            (int(so) * so_points))


In [231]:
# If points is selected

df_hitters_merged['FantasyPoints_Hitting'] = df_hitters_merged.apply(lambda x: calculate_hitting(x.PA,
                                                                   x.H,
                                                                   x.S, 
                                                                   x.D,                       
                                                                   x['T'],
                                                                   x.HR,                                                               
                                                                   x.SO,
                                                                   x.BB,
                                                                   x.HBP,
                                                                   x.SB,
                                                                   x.CS,                          
                                                                   x.R,
                                                                   x.RBI,
                                                                   custom_points[0],
                                                                   custom_points[1],
                                                                   custom_points[2],
                                                                   custom_points[3],
                                                                   custom_points[4], 
                                                                   custom_points[5],
                                                                   custom_points[6],
                                                                   custom_points[7],
                                                                   custom_points[8],
                                                                   custom_points[9],
                                                                   custom_points[10],
                                                                   custom_points[11], 
                                                                   custom_points[12]),axis=1)


In [233]:
budget = 260
min_bid = 1
teams = 12
bat_split = 0.7
p_split = 1 - bat_split
player_universe = ['MLB', 'AL', 'NL']


In [234]:
roster_C = 1
roster_1B = 1
roster_2B = 1
roster_3B = 1
roster_SS = 1
roster_OF = 3
roster_DH = 0
roster_UTIL = 2
roster_MI = 0
roster_CI = 0
roster_SP = 2
roster_RP = 2
roster_P = 4
roster_B = 5

In [237]:
# Pare down the batters list to fit with the teams/roster req's
df_list = []

rosters = [roster_C, roster_2B, roster_SS, roster_OF, roster_3B, roster_1B, roster_DH, roster_MI, roster_CI, roster_UTIL]
positions = ['C', '2B', 'SS', 'OF', '3B', '1B', 'DH', 'MI', 'CI', 'UTIL']
df_pared = df_hitters_merged

i = 0
for pos in positions:
    sort_by = 'HR'
    if league_format == 'Points':
        sort_by = 'FantasyPoints_Hitting'
    if i == 0:
        df_pos = df_hitters_merged[df_hitters_merged['Pos'].apply(lambda x: pos in x)].sort_values(by=[sort_by], ascending=False).reset_index(drop=True)
    else:
        df_pos = df_pared[df_pared['Pos'].apply(lambda x: pos in x)].sort_values(by=[sort_by], ascending=False).reset_index(drop=True)
    num_pos = teams * rosters[i]
    df_roster = df_pos.head(num_pos)
    df_roster['selected_pos'] = pos
    df_list.append(df_roster)
    chosen_players = df_roster['idfangraphs'].to_list()
    df_pared['selected'] = df_pared.apply(lambda x: 1 if x.idfangraphs in chosen_players else 0, axis=1)
    df_pared = df_pared[df_pared['selected'] < 1]
    i+=1
    
draft_pool = pd.concat(df_list)


FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting
FantasyPoints_Hitting


/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/2834021479.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roster['selected_pos'] = pos
/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/2834021479.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pared['selected'] = df_pared.apply(lambda x: 1 if x.idfangraphs in chosen_players else 0, axis=1)


In [172]:
league_avg = sum(draft_pool.H.to_list()) / sum(draft_pool.AB.to_list())
league_obp = (sum(draft_pool.H.to_list()) + sum(draft_pool.BB.to_list())) / sum(draft_pool.PA.to_list())
league_slg = sum(draft_pool.TB.to_list()) / sum(draft_pool.AB.to_list())
league_woba = ((sum(draft_pool.BB.to_list()) * .692) + (sum(draft_pool.S.to_list()) * .879) + (sum(draft_pool.D.to_list()) * 1.242) + (sum(draft_pool['T'].to_list()) * 1.568) + (sum(draft_pool.HR.to_list()) * 2.007)) / (sum(draft_pool.AB.to_list()) + sum(draft_pool.BB.to_list()))

draft_pool['HaAVG'] = draft_pool.apply(lambda x: x.H - (x.AB * league_avg), axis=1)
draft_pool['OBaAVG'] = draft_pool.apply(lambda x: (x.H + x.BB) - (x.PA * league_obp), axis=1)
draft_pool['TBaAVG'] = draft_pool.apply(lambda x: x.TB - (x.AB * league_slg), axis=1)
draft_pool['OPSaAVG'] = draft_pool.apply(lambda x: x.OBaAVG + x.TBaAVG, axis=1)
draft_pool['wOBAaAVG'] = draft_pool.apply(lambda x: ((x.BB * .692) + (x.S * .879) + (x.D * 1.242) + (x['T'] * 1.568) + (x.HR * 2.007)) - ((x.BB + x.AB) * league_woba), axis=1)



In [173]:
all_cats = batting_categories + ['HaAVG', 'OBaAVG', 'TBaAVG', 'OPSaAVG', 'wOBAaAVG']
agg_stats = draft_pool[all_cats]
analysis = agg_stats.describe()

In [174]:
for cat in all_cats:
    draft_pool['m' + cat] = draft_pool.apply(lambda x: (x[cat] - analysis.at['mean', cat]) / (analysis.at['std', cat]), axis=1)


In [175]:
# batting_categories = ['AVG', 'RBI', 'R', 'SB', 'HR', 'OBP', 'SLG', 'OPS', 
# 'H', 'SO', 'S', 'D', 'T', 'TB', 'BB', 'RBI+R', 'xBH', 'SB-CS', 'wOBA']

batting_stats_dict = {
    'AVG': 'mHaAVG',
    'RBI': 'mRBI',
    'R': 'mR',
    'SB': 'mSB',
    'HR': 'mHR',
    'OBP': 'mOBaAVG',
    'SLG': 'mTBaAVG',
    'OPS': 'mOPSaAVG',
    'H': 'mH',
    'SO': 'mSO',
    'S': 'mS',
    'D': 'mD',
    'T': 'mT',
    'TB': 'mTB',
    'BB': 'mBB',
    'RBI+R': 'mRBI+R',
    'xBH': 'mxBH',
    'SB-CS': 'mSB-CS',
    'wOBA': 'mwOBA'
}

In [186]:
# Pare down this dataframe to only relevant stat columns
if league_format == 'Roto':
    selected_mstats = []
    for stat in selected_batting_stats:
        mstat = batting_stats_dict[stat]
        selected_mstats.append(mstat)
    draft_stats = draft_pool[['idfangraphs', 'mlbid', 'Name', 'Year', 'Team', 'age', 'selected_pos'] + selected_mstats]
    draft_stats['mV'] = draft_stats[selected_mstats].sum(axis=1)

/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/2072787707.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_stats['mV'] = draft_stats[selected_mstats].sum(axis=1)


,idfangraphs,mlbid,Name,Year,Team,age,selected_pos,mHaAVG,mRBI,mR,mSB,mHR,mV
0,7304,521692.0,Salvador Perez,2022,KCR,32,C,0.932054,2.372930,0.162548,-0.682628,2.677349,5.462252
1,19197,669257.0,Will Smith,2022,LAD,27,C,0.272963,0.854969,0.162548,-0.511615,0.425899,1.204764
2,11442,596142.0,Gary Sanchez,2022,NYY,30,C,-2.117707,-0.841575,-1.291832,-0.853642,0.425899,-4.678857
3,13265,572287.0,Mike Zunino,2022,TBR,31,C,-2.237992,-1.109451,-1.120729,-0.853642,0.200754,-5.121059
4,15161,641598.0,Mitch Garver,2022,MIN,31,C,-0.609566,-1.377326,-1.206281,-0.853642,-0.474681,-4.521495
5,11368,518735.0,Yasmani Grandal,2022,CHW,34,C,-0.871188,-0.573700,-0.436314,-0.682628,-0.474681,-3.038511
6,11609,575929.0,Willson Contreras,2022,CHC,30,C,-0.546709,-0.662992,0.076997,-0.340602,-0.699826,-2.173132
7,11739,592663.0,J.T. Realmuto,2022,PHI,31,C,0.163838,0.140635,0.162548,0.685479,-0.924970,0.227529
8,19352,669221.0,Sean Murphy,2022,OAK,28,C,-1.187885,-1.466618,-1.206281,-0.853642,-1.150115,-5.864541
9,13620,608348.0,Carson Kelly,2022,ARI,28,C,-0.701800,-1.109451,-1.805143,-0.853642,-1.375260,-5.845296


In [187]:
selected_positions = list(set(draft_stats['selected_pos'].to_list()))

['C', 'SS', '1B', 'OF', '3B', 'UTIL', '2B']

In [189]:
positional_adjustment = {}
for pos in selected_positions:
    adj = min(draft_stats[draft_stats['selected_pos'] == pos]['mV'])
    positional_adjustment[pos] = -adj

{'C': 7.241345593326959,
 'SS': 1.4493764115133652,
 '1B': 1.329118409287015,
 'OF': 5.413918624719857,
 '3B': 3.233620522565911,
 'UTIL': 5.670920190955005,
 '2B': 5.136898624381263}

In [191]:
draft_stats['mPos'] = draft_stats.apply(lambda x: positional_adjustment[x.selected_pos], axis=1)
draft_stats['m$'] = draft_stats.apply(lambda x: x.mV + x.mPos, axis=1)

/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/3134932345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_stats['mPos'] = draft_stats.apply(lambda x: positional_adjustment[x.selected_pos], axis=1)
/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/3134932345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_stats['m$'] = draft_stats.apply(lambda x: x.mV + x.mPos, axis=1)


,idfangraphs,mlbid,Name,Year,Team,age,selected_pos,mHaAVG,mRBI,mR,mSB,mHR,mV,mPos,m$
0,7304,521692.0,Salvador Perez,2022,KCR,32,C,0.932054,2.372930,0.162548,-0.682628,2.677349,5.462252,7.241346,12.703598
1,19197,669257.0,Will Smith,2022,LAD,27,C,0.272963,0.854969,0.162548,-0.511615,0.425899,1.204764,7.241346,8.446110
2,11442,596142.0,Gary Sanchez,2022,NYY,30,C,-2.117707,-0.841575,-1.291832,-0.853642,0.425899,-4.678857,7.241346,2.562488
3,13265,572287.0,Mike Zunino,2022,TBR,31,C,-2.237992,-1.109451,-1.120729,-0.853642,0.200754,-5.121059,7.241346,2.120287
4,15161,641598.0,Mitch Garver,2022,MIN,31,C,-0.609566,-1.377326,-1.206281,-0.853642,-0.474681,-4.521495,7.241346,2.719850


In [204]:
total_value = sum(draft_stats['m$'])
dollars_spent = (budget * teams * bat_split) - (teams * sum(rosters))
dollars_per_value = dollars_spent / total_value

3.741248647040715

In [206]:
draft_stats['auction$'] = draft_stats.apply(lambda x: round((x['m$'] * dollars_per_value) + 1, 1), axis=1)
draft_stats

/var/folders/ly/7vdr23qj7yn96w59b90h_zmm0000gn/T/ipykernel_46179/1967993689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_stats['$'] = draft_stats.apply(lambda x: round((x['m$'] * dollars_per_value) + 1, 1), axis=1)


,idfangraphs,mlbid,Name,Year,Team,age,selected_pos,mHaAVG,mRBI,mR,mSB,mHR,mV,mPos,m$,$
0,7304,521692.0,Salvador Perez,2022,KCR,32,C,0.932054,2.372930,0.162548,-0.682628,2.677349,5.462252,7.241346,12.703598,48.5
1,19197,669257.0,Will Smith,2022,LAD,27,C,0.272963,0.854969,0.162548,-0.511615,0.425899,1.204764,7.241346,8.446110,32.6
2,11442,596142.0,Gary Sanchez,2022,NYY,30,C,-2.117707,-0.841575,-1.291832,-0.853642,0.425899,-4.678857,7.241346,2.562488,10.6
3,13265,572287.0,Mike Zunino,2022,TBR,31,C,-2.237992,-1.109451,-1.120729,-0.853642,0.200754,-5.121059,7.241346,2.120287,8.9
4,15161,641598.0,Mitch Garver,2022,MIN,31,C,-0.609566,-1.377326,-1.206281,-0.853642,-0.474681,-4.521495,7.241346,2.719850,11.2
5,11368,518735.0,Yasmani Grandal,2022,CHW,34,C,-0.871188,-0.573700,-0.436314,-0.682628,-0.474681,-3.038511,7.241346,4.202835,16.7
6,11609,575929.0,Willson Contreras,2022,CHC,30,C,-0.546709,-0.662992,0.076997,-0.340602,-0.699826,-2.173132,7.241346,5.068214,20.0
7,11739,592663.0,J.T. Realmuto,2022,PHI,31,C,0.163838,0.140635,0.162548,0.685479,-0.924970,0.227529,7.241346,7.468875,28.9
8,19352,669221.0,Sean Murphy,2022,OAK,28,C,-1.187885,-1.466618,-1.206281,-0.853642,-1.150115,-5.864541,7.241346,1.376805,6.2
9,13620,608348.0,Carson Kelly,2022,ARI,28,C,-0.701800,-1.109451,-1.805143,-0.853642,-1.375260,-5.845296,7.241346,1.396049,6.2


In [251]:
# If points league

minimum = min(draft_pool['FantasyPoints_Hitting'])
maximum = max(draft_pool['FantasyPoints_Hitting'])
std = draft_pool['FantasyPoints_Hitting'].std()
mean = draft_pool['FantasyPoints_Hitting'].mean()
def max_std(std, mean, maximum):
    i = 1
    j = 0
    while i > 0:
        if (maximum > mean + (j * std)) & (maximum < mean + (i * std)):
            return i
        else:
            i += 1
            j += 1

sd_above = max_std(std, mean, maximum)

tot_players = teams * sum(rosters)
avg_budget = (budget * teams * bat_split) / tot_players

In [252]:
avg_budget

18.2

In [253]:
m = interp1d([minimum,maximum],[min_bid,avg_budget*sd_above])
draft_pool['auction$'] = round(draft_pool['FantasyPoints_Hitting'].apply(m), 1)

draft_pool

,idfangraphs,mlbid,Name,Year,Team,age,wOBA,AVG,OBP,SLG,PA,AB,R,H,S,D,T,HR,RBI,SB,CS,BB,SO,allpos,Position,MI,CI,Pos,OPS,TB,RBI+R,xBH,SB-CS,HBP,FantasyPoints_Hitting,selected_pos,selected,auction$
0,7304,521692.0,Salvador Perez,2022,KCR,32,0.362,0.277,0.321,0.526,639,591,83,164,95,30,0,39,108,3,1,31,155,C,[C],[],[],"[C, UTIL]",0.847,311,191,69,2,0,1264.7,C,NaN,43.7
1,19197,669257.0,Will Smith,2022,LAD,27,0.381,0.266,0.363,0.509,579,485,83,129,72,27,2,29,91,4,0,66,117,C,[C],[],[],"[C, UTIL]",0.872,248,174,58,4,0,1163.8,C,NaN,31.3
2,11739,592663.0,J.T. Realmuto,2022,PHI,31,0.346,0.263,0.334,0.455,604,536,83,141,87,28,3,23,83,11,2,50,138,C/DH,"[C, DH]",[],[],"[C, DH, UTIL]",0.789,244,166,54,9,0,1126.0,C,NaN,26.7
3,11368,518735.0,Yasmani Grandal,2022,CHW,34,0.364,0.238,0.366,0.450,563,458,76,109,63,20,1,25,75,3,1,92,135,C/DH,"[C, DH]",[],[],"[C, DH, UTIL]",0.816,206,151,46,2,0,1074.3,C,NaN,20.3
4,11609,575929.0,Willson Contreras,2022,CHC,30,0.347,0.248,0.342,0.441,602,517,82,128,77,26,1,24,74,5,4,58,154,C/DH,"[C, DH]",[],[],"[C, DH, UTIL]",0.783,228,156,51,1,0,1061.0,C,NaN,18.7
5,19918,662139.0,Daulton Varsho,2022,ARI,26,0.324,0.238,0.317,0.416,546,483,72,115,67,27,4,17,61,10,2,53,125,C/OF/DH,"[C, OF, DH]",[],[],"[C, OF, DH, UTIL]",0.733,201,133,48,8,0,955.1,C,NaN,5.7
6,17988,663886.0,Tyler Stephenson,2022,CIN,26,0.358,0.280,0.359,0.448,532,464,73,130,87,25,1,17,66,2,1,53,109,C,[C],[],[],"[C, UTIL]",0.807,208,139,43,1,0,951.1,C,NaN,5.2
7,11442,596142.0,Gary Sanchez,2022,NYY,30,0.320,0.209,0.301,0.430,542,474,66,99,52,16,1,29,72,2,1,55,152,C/DH,"[C, DH]",[],[],"[C, DH, UTIL]",0.731,203,138,46,1,0,941.4,C,NaN,4.1
8,15161,641598.0,Mitch Garver,2022,MIN,31,0.349,0.244,0.331,0.471,499,435,67,106,58,22,1,25,66,2,1,53,140,C,[C],[],[],"[C, UTIL]",0.802,205,133,48,1,0,931.9,C,NaN,2.9
9,19352,669221.0,Sean Murphy,2022,OAK,28,0.332,0.231,0.326,0.425,540,468,67,108,61,25,0,22,65,2,1,59,132,C,[C],[],[],"[C, UTIL]",0.751,199,132,47,1,0,930.0,C,NaN,2.7


In [255]:
draft_pool.to_csv('draft_stats_yahoo_points_v1.csv')